In [3]:
import pandas as pd
import numpy as np
from icecream import ic
'''
 - (1) pivot()
 90도 회전(rotation)
 컬럼 -> 인덱스로 변환

 - (2) stack(), unstack()
 stack이 (위에서 아래로 길게, 높게) 쌓는 것이면,
 unstack은 쌓은 것을 옆으로 늘어놓는것(왼쪽에서 오른쪽으로 넓게) 라고 연상이 될 것입니다.



 - (3) melt()

 - (4) wide_to_long()

 - (5) pd.crosstab()

'''

'\n - (1) pivot(), pd.pivot_table()\n\n - (2) stack(), unstack()\n\n - (3) melt()\n\n - (4) wide_to_long()\n\n - (5) pd.crosstab()\n\n'

In [6]:
# 30 다음 객체를 customer_id 를 인덱스로하고 product_code 를 컬럼으로, purchare_amount 를 값으로, 재구성하시오
df = pd.DataFrame({"customer_id":['kim','lee','park','song','yoon','kang','tak','ryu','jang'],
               "product_code":['com','phone','tv','com','phone','tv','com','phone','tv'],
               "grade":['A','A','A','A','A','A','B','B','B'],
               "purchase_amount":[30,10,0,40,15,30,0,0,10]})
ic(df)
df = df.pivot(index="customer_id", columns="product_code", values="purchase_amount")
ic(df)

ic| df:   customer_id product_code grade  purchase_amount
        0         kim          com     A               30
        1         lee        phone     A               10
        2        park           tv     A                0
        3        song          com     A               40
        4        yoon        phone     A               15
        5        kang           tv     A               30
        6         tak          com     B                0
        7         ryu        phone     B                0
        8        jang           tv     B               10


product_code   com  phone    tv
customer_id                    
jang           NaN    NaN  10.0
kang           NaN    NaN  30.0
kim           30.0    NaN   NaN
lee            NaN   10.0   NaN
park           NaN    NaN   0.0
ryu            NaN    0.0   NaN
song          40.0    NaN   NaN
tak            0.0    NaN   NaN
yoon           NaN   15.0   NaN


In [7]:
# 31 다음 객체를 customer_id, grade 를 인덱스로하고 product_code 를 컬럼으로, purchare_amount 를 값으로, 재구성하시오

df = pd.DataFrame({"customer_id":['kim','lee','park','song','yoon','kang','tak','ryu','jang'],
               "product_code":['com','phone','tv','com','phone','tv','com','phone','tv'],
               "grade":['A','A','A','A','A','A','B','B','B'],
               "purchase_amount":[30,10,0,40,15,30,0,0,10]})
df = df.pivot(index=["customer_id","grade"],  columns="product_code", values="purchase_amount")
ic(df)

ic| df: product_code        com  phone    tv
        customer_id grade                   
        jang        B       NaN    NaN  10.0
        kang        A       NaN    NaN  30.0
        kim         A      30.0    NaN   NaN
        lee         A       NaN   10.0   NaN
        park        A       NaN    NaN   0.0
        ryu         B       NaN    0.0   NaN
        song        A      40.0    NaN   NaN
        tak         B       0.0    NaN   NaN
        yoon        A       NaN   15.0   NaN


,product_code,com,phone,tv
customer_id,grade,,,
jang,B,NaN,NaN,10.0
kang,A,NaN,NaN,30.0
kim,A,30.0,NaN,NaN
lee,A,NaN,10.0,NaN
park,A,NaN,NaN,0.0
ryu,B,NaN,0.0,NaN
song,A,40.0,NaN,NaN
tak,B,0.0,NaN,NaN
yoon,A,NaN,15.0,NaN


In [27]:
# 32
df = pd.DataFrame(np.arange(16).reshape(4,4),
                index = pd.MultiIndex.from_tuples([('kim','2019'),('kim','2020'),('lee','2019'),('lee','2020')]),
                columns = ['com','phone','tv','notebook'])
ic(df)
# ic(type(df.stack())) pandas.core.series.Series
# 칼럼의 level은 1개 밖에 없으므로 stack(level=-1) 을 별도로 명기하지 않아도 됩니다.
df = df.stack(level=-1, dropna=True)
ic(df)
df = df.unstack(level=0)
ic(df.unstack())
'''
결측값이 있는 데이터셋을 stack() 할 때 결측값을 제거할지(dropna=True),
아니면 결측값을 NaN으로 유지할지(dropna=False) 설정
unstack(level=-1), unstack(level=0), unstack(level=1) 별로
어떤 level이 칼럼으로 이동해서 unstack() 되는지 체크
'''
df = df.reset_index()
df.rename(columns={'level_0':'customer_id','level_1':'year'}, inplace=True)
ic(df)




ic| df:           com  phone  tv  notebook
        kim 2019    0      1   2         3
            2020    4      5   6         7
        lee 2019    8      9  10        11
            2020   12     13  14        15
ic| df: kim  2019  com          0
                   phone        1
                   tv           2
                   notebook     3
             2020  com          4
                   phone        5
                   tv           6
                   notebook     7
        lee  2019  com          8
                   phone        9
                   tv          10
                   notebook    11
             2020  com         12
                   phone       13
                   tv          14
                   notebook    15
        dtype: int32
ic| df.unstack():      kim                   lee                   
                       com phone tv notebook com phone  tv notebook
                  2019   0     1  2        3   8     9  10       11
                

,customer_id,year,kim,lee
0,2019,com,0,8
1,2019,phone,1,9
2,2019,tv,2,10
3,2019,notebook,3,11
4,2020,com,4,12
5,2020,phone,5,13
6,2020,tv,6,14
7,2020,notebook,7,15


In [28]:
'''
melt() 는 ID 변수를 기준으로 원래 데이터셋에 있던 여러개의 칼럼 이름을
'variable' 칼럼에 위에서 아래로 길게 쌓아놓고,
'value' 칼럼에 ID와 variable에 해당하는 값을
넣어주는 식으로 데이터를 재구조화
'''
df = pd.DataFrame({"customer_id":['kim','kim','lee','lee'],
               "product_code":['com','phone','tv','tab'],
               "purchase_count":[1,2,3,4],
               "purchase_amount":[100,200,300,400]})
ic(df)

# (1) pd.melt(data, id_vars=['id1', 'id2', ...]) 를 사용한 데이터 재구조화


# (2) pd.melt() 의 variable 이름, value 이름 부여하기 : var_name, value_name



ic| df:   customer_id product_code  purchase_count  purchase_amount
        0         kim          com               1              100
        1         kim        phone               2              200
        2         lee           tv               3              300
        3         lee          tab               4              400


,customer_id,product_code,purchase_count,purchase_amount
0,kim,com,1,100
1,kim,phone,2,200
2,lee,tv,3,300
3,lee,tab,4,400
